## Lecture 14 code-along notebook


Results of comparing different convolution implementations:

tensors shapes:
```python
X.shape = (10, 32, 32, 8)
weight.shape = (3, 3, 8, 16)
```
  
timings:

|Convolution implementation|Time to run|
|-|-|
|Naive|10.9 s|
|Multiple matrix multiplications|6.56 ms|
|im2col|4.32 ms|
|Pytorch, CPU|2.06 ms|

In [1]:
import numpy as np
import ctypes

import torch
import torch.nn as nn

In [2]:
def get_np_underlying_memory(arr: np.ndarray):
    return np.frombuffer(ctypes.string_at(arr.ctypes.data, arr.nbytes), dtype=arr.dtype, count=arr.size)

In [3]:
Z = np.random.rand(10, 32, 32, 8)
weight = np.random.rand(3, 3, 8, 16)

## Reference implementation. Pytorch, CPU

In [4]:
def conv_reference(Z, weight):
    Z_torch = torch.tensor(Z).permute(0, 3, 1, 2)
    w_torch = torch.tensor(weight).permute(3, 2, 0, 1)
    out = nn.functional.conv2d(Z_torch, w_torch)
    return out.permute(0, 2, 3, 1).contiguous().numpy()

In [5]:
out_ref = conv_reference(Z, weight)
print(out_ref.shape)

(10, 30, 30, 16)


In [6]:
%%timeit
conv_reference(Z, weight)

2.06 ms ± 128 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Naive implementation

In [7]:
def conv_naive(Z, weight):
    N, H, W, C_in = Z.shape
    K, _, _, C_out = weight.shape
    assert K % 2 == 1

    out = np.zeros(shape=(N, H - K + 1, W - K + 1, C_out))

    # batch
    for ib in range(N):
        # image
        for i in range(H - K + 1):
            for j in range(W - K + 1):
                # channels
                for icin in range(C_in):
                    for icout in range(C_out):
                        # kernel
                        for ik in range(0, K):
                            for jk in range(0, K):
                                out[ib, i, j, icout] += Z[ib, i + ik, j + jk, icin] * weight[ik, jk, icin, icout]
    
    return out

In [8]:
out_naive = conv_naive(Z, weight)
print(out_naive.shape)

(10, 30, 30, 16)


In [9]:
np.linalg.norm(out_ref - out_naive)

2.026778783270082e-12

In [10]:
%%time 
_ = conv_naive(Z, weight)

CPU times: total: 11.2 s
Wall time: 10.9 s


## Convolution as a number of matrix multiplications

In [11]:
def conv_matrix_mult(Z, weight):
    N, H, W, C_in = Z.shape
    K, _, _, C_out = weight.shape
    assert K % 2 == 1

    out = np.zeros(shape=(N, H - K + 1, W - K + 1, C_out))

    for i in range(K):
        for j in range(K):
            out += Z[:, i:i + H - K + 1, j:j + W - K + 1, :] @ weight[i, j, :, :]

    return out

In [12]:
out_mm = conv_matrix_mult(Z, weight)
print(out_mm.shape)

(10, 30, 30, 16)


In [13]:
np.linalg.norm(out_ref - out_mm)

1.7667661555878716e-12

In [14]:
%%timeit 
x = conv_matrix_mult(Z, weight)

6.56 ms ± 675 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## im2col exploration
* step-by-step into im2col

In [15]:
m, n, k = 5, 5, 3
x = np.arange(m * n, dtype=np.int32).reshape(m, n)
print(x.shape, np.array(x.strides) / 4, x.size)
print(x)

(5, 5) [5. 1.] 25
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]


In [16]:
get_np_underlying_memory(x)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])

In [17]:
x1 = np.lib.stride_tricks.as_strided(
    x, 
    shape=(m - k + 1, n - k + 1, k, k),
    strides=np.array((n, 1, n, 1)) * 4
)
print(x1.shape, np.array(x1.strides) / 4, x1.size)

(3, 3, 3, 3) [5. 1. 5. 1.] 81


In [18]:
get_np_underlying_memory(x1)  # accesses unallocated memory because array is not contiguous and has wrong size

array([          0,           1,           2,           3,           4,
                 5,           6,           7,           8,           9,
                10,          11,          12,          13,          14,
                15,          16,          17,          18,          19,
                20,          21,          22,          23,          24,
         929182050, -1499276895, -1878803968,  1055110768,         754,
        1055110752,         754,  1055111488,         754,  1055111472,
               754,  1056441856,         754,  1056441840,         754,
        1056449776,         754,  1056449760,         754,  1056445696,
               754,  1056445680,         754,  1056695040,         754,
        1056695024,         754,           0,           0, -1498687064,
       -1878803712,  -171990272,       32765,  -131123856,       32765,
        1093899008,         754,  1093898992,         754,  1919972974,
         980643177,  1970435130,       25454,          14,      

In [19]:
x2 = np.ascontiguousarray(x1)
print(x2.shape, np.array(x2.strides) / 4, x2.size)

(3, 3, 3, 3) [27.  9.  3.  1.] 81


In [20]:
x2

array([[[[ 0,  1,  2],
         [ 5,  6,  7],
         [10, 11, 12]],

        [[ 1,  2,  3],
         [ 6,  7,  8],
         [11, 12, 13]],

        [[ 2,  3,  4],
         [ 7,  8,  9],
         [12, 13, 14]]],


       [[[ 5,  6,  7],
         [10, 11, 12],
         [15, 16, 17]],

        [[ 6,  7,  8],
         [11, 12, 13],
         [16, 17, 18]],

        [[ 7,  8,  9],
         [12, 13, 14],
         [17, 18, 19]]],


       [[[10, 11, 12],
         [15, 16, 17],
         [20, 21, 22]],

        [[11, 12, 13],
         [16, 17, 18],
         [21, 22, 23]],

        [[12, 13, 14],
         [17, 18, 19],
         [22, 23, 24]]]])

In [21]:
get_np_underlying_memory(x2)

array([ 0,  1,  2,  5,  6,  7, 10, 11, 12,  1,  2,  3,  6,  7,  8, 11, 12,
       13,  2,  3,  4,  7,  8,  9, 12, 13, 14,  5,  6,  7, 10, 11, 12, 15,
       16, 17,  6,  7,  8, 11, 12, 13, 16, 17, 18,  7,  8,  9, 12, 13, 14,
       17, 18, 19, 10, 11, 12, 15, 16, 17, 20, 21, 22, 11, 12, 13, 16, 17,
       18, 21, 22, 23, 12, 13, 14, 17, 18, 19, 22, 23, 24])

In [22]:
x3 = x2.reshape(((m - k + 1) * (n - k + 1), k * k))
print(x3.shape, np.array(x3.strides) / 4, x3.size)

(9, 9) [9. 1.] 81


In [23]:
x3

array([[ 0,  1,  2,  5,  6,  7, 10, 11, 12],
       [ 1,  2,  3,  6,  7,  8, 11, 12, 13],
       [ 2,  3,  4,  7,  8,  9, 12, 13, 14],
       [ 5,  6,  7, 10, 11, 12, 15, 16, 17],
       [ 6,  7,  8, 11, 12, 13, 16, 17, 18],
       [ 7,  8,  9, 12, 13, 14, 17, 18, 19],
       [10, 11, 12, 15, 16, 17, 20, 21, 22],
       [11, 12, 13, 16, 17, 18, 21, 22, 23],
       [12, 13, 14, 17, 18, 19, 22, 23, 24]])

* in numpy, we can skip explicit `ascontiguousarray()` call as numpy will perform it implicitly during `reshape()`

In [24]:
x3_2 = x1.reshape(((m - k + 1) * (n - k + 1), k * k))
print(x3_2.shape, np.array(x3_2.strides) / 4, x3_2.size)

(9, 9) [9. 1.] 81


In [25]:
x3_2

array([[ 0,  1,  2,  5,  6,  7, 10, 11, 12],
       [ 1,  2,  3,  6,  7,  8, 11, 12, 13],
       [ 2,  3,  4,  7,  8,  9, 12, 13, 14],
       [ 5,  6,  7, 10, 11, 12, 15, 16, 17],
       [ 6,  7,  8, 11, 12, 13, 16, 17, 18],
       [ 7,  8,  9, 12, 13, 14, 17, 18, 19],
       [10, 11, 12, 15, 16, 17, 20, 21, 22],
       [11, 12, 13, 16, 17, 18, 21, 22, 23],
       [12, 13, 14, 17, 18, 19, 22, 23, 24]])

### refactor im2col to be a function

In [26]:
def im2col_2d(arr, m, n, k):
    out1 = np.lib.stride_tricks.as_strided(
        arr, 
        shape=(m - k + 1, n - k + 1, k, k),
        strides=np.array((n, 1, n, 1)) * 4
    )
    # numpy makes array contiguous in memory before reshape if needed (like in this case)
    out2 = out1.reshape(((m - k + 1) * (n - k + 1), k * k))
    return out2

In [27]:
x

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [28]:
im2col_2d(x, 5, 5, 3)

array([[ 0,  1,  2,  5,  6,  7, 10, 11, 12],
       [ 1,  2,  3,  6,  7,  8, 11, 12, 13],
       [ 2,  3,  4,  7,  8,  9, 12, 13, 14],
       [ 5,  6,  7, 10, 11, 12, 15, 16, 17],
       [ 6,  7,  8, 11, 12, 13, 16, 17, 18],
       [ 7,  8,  9, 12, 13, 14, 17, 18, 19],
       [10, 11, 12, 15, 16, 17, 20, 21, 22],
       [11, 12, 13, 16, 17, 18, 21, 22, 23],
       [12, 13, 14, 17, 18, 19, 22, 23, 24]])

In [29]:
def im2col_2d_torch(arr: torch.Tensor, m, n, k):
    out1 = arr.as_strided(
        size=(m - k + 1, n - k + 1, k, k),
        stride=(n, 1, n, 1)
    )
    # pytorch will fail if we try to change shape with .view() on non-contiguous array.
    # we can use either .contiguous().view() or .reshape()
    out2 = out1.contiguous().view(((m - k + 1) * (n - k + 1), k * k))
    return out2

In [30]:
x_t = torch.Tensor(x)
print(x_t.size(), x_t.stride())

torch.Size([5, 5]) (5, 1)


In [31]:
im2col_2d_torch(x_t, m, n, k)

tensor([[ 0.,  1.,  2.,  5.,  6.,  7., 10., 11., 12.],
        [ 1.,  2.,  3.,  6.,  7.,  8., 11., 12., 13.],
        [ 2.,  3.,  4.,  7.,  8.,  9., 12., 13., 14.],
        [ 5.,  6.,  7., 10., 11., 12., 15., 16., 17.],
        [ 6.,  7.,  8., 11., 12., 13., 16., 17., 18.],
        [ 7.,  8.,  9., 12., 13., 14., 17., 18., 19.],
        [10., 11., 12., 15., 16., 17., 20., 21., 22.],
        [11., 12., 13., 16., 17., 18., 21., 22., 23.],
        [12., 13., 14., 17., 18., 19., 22., 23., 24.]])

### Convolution

In [32]:
w = np.arange(k * k).reshape(k, k)
w

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [33]:
(im2col_2d(x, m, n, k) @ w.reshape(k * k)).reshape(m - k + 1, n - k + 1)

array([[312, 348, 384],
       [492, 528, 564],
       [672, 708, 744]])

## im2col convolution. Multi-channel case

In [34]:
def im2col(arr: np.ndarray, K):
    B, H, W, Cin = arr.shape
    Bs, Hs, Ws, Cs = arr.strides

    out = np.lib.stride_tricks.as_strided(
        arr, 
        shape=(B, H - K + 1, W - K + 1, K, K, Cin),
        strides=(Bs, Hs, Ws, Hs, Ws, Cs)
    )
    # numpy makes array contiguous in memory before reshape if needed - like in this case.
    # here we not only change the shape, but also duplicate needed values of input tensor.
    # thus, underlying data copy is required.
    out = out.reshape(-1, K * K * Cin)
    
    return out

In [35]:
def conv_im2col(Z, weight):
    N, H, W, C_in = Z.shape
    K, _, _, C_out = weight.shape
    assert K % 2 == 1

    Z_im2col = im2col(Z, K)
    out = Z_im2col @ weight.reshape(-1, C_out)
    out = out.reshape(N, H - K + 1, W - K + 1, C_out)

    return out

In [36]:
out_im2col = conv_im2col(Z, weight)
print(out_im2col.shape)

(10, 30, 30, 16)


In [37]:
np.linalg.norm(out_ref - out_im2col)

8.896161051565601e-13

In [38]:
%%timeit
conv_im2col(Z, weight)

4.32 ms ± 123 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
